In [1]:
import pandas as pd
import numpy as np

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, merge, Dense, TimeDistributed, Dropout
from keras.layers import Bidirectional, concatenate, SpatialDropout1D
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm
from numpy import dot


import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from keras.layers import add

data = pd.read_csv("ner_dataset.csv", encoding="latin1")

Using TensorFlow backend.


In [2]:
data = data.fillna(method="ffill")
data.tail(5)

,Sentence #,Word,POS,Tag
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [3]:
wordwa = list(set(data["Word"].values))
words = []

for w in wordwa :
    w = str(w)
    w = w.lower()
    words.append(w)
print(words[0:10])
n_words = len(words); 
n_words

['oman', 'smalls', 'aiko', 'zazai', 'singly', 'proclamation', 'ceausescu', 'arauca', 'goss', 'cabinet']


35178

In [4]:
chars = set([w_i for w in words for w_i in w])
n_chars = len(chars)
print(n_chars)

72


In [5]:
char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["UNK"] = 1
char2idx["PAD"] = 0

In [6]:
print(char2idx['k'])
print(char2idx['-'])

46
58


In [7]:
max_word_length = 25
embedded_char_vector_length = 74 # Length of character dictionary
char_feature_output = 10

In [8]:
X_char = []
for word in words :
    l1 = []
    #if (len(word) > max_word_length) :
    #    print(len(word))
    for i in range(max_word_length):
        try:
            l1.append(char2idx.get(word[i]))
        except:
            l1.append(char2idx.get("PAD"))    
    
    X_char.append(l1)

In [9]:
print(X_char[0:10])

[[63, 9, 3, 71, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [11, 9, 3, 59, 59, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [3, 24, 46, 63, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [35, 3, 35, 3, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [11, 24, 71, 39, 59, 31, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [47, 43, 63, 25, 59, 3, 9, 3, 8, 24, 63, 71, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [25, 64, 3, 57, 11, 64, 11, 25, 57, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [3, 43, 3, 57, 25, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [39, 63, 11, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [25, 3, 54, 24, 71, 64, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [10]:
char_input = Input(shape=(max_word_length,), dtype='float32', name='char_input')
char_input1 = Embedding(74, embedded_char_vector_length, input_length=max_word_length)(char_input)
char_input2 = Dropout(0.2)(char_input1)
'''lstm_out_forward = LSTM(char_feature_output, dropout=0.2, recurrent_dropout=0.2)(char_input2)
lstm_out_backward = LSTM(char_feature_output, go_backwards=True, dropout=0.2, recurrent_dropout=0.2)(char_input2)'''

merged = (Bidirectional(LSTM(units=5, return_sequences=False,trainable = False,
                                recurrent_dropout=0.5)))(char_input2)
#merged = merge([lstm_out_forward, lstm_out_backward], mode='concat', concat_axis=1)

#merged = add([lstm_out_forward, lstm_out_backward])
#model = Model(input=[char_input], output=[merged])
model = Model(char_input, merged)

In [11]:
'''X1 = np.array([[1,0,0],[0,0,1],[0,1,0],[0,0,0],[0,0,0],[0,0,0]])
train_char_vectors = model.predict(X1)

print(train_char_vectors)'''

'X1 = np.array([[1,0,0],[0,0,1],[0,1,0],[0,0,0],[0,0,0],[0,0,0]])\ntrain_char_vectors = model.predict(X1)\n\nprint(train_char_vectors)'

In [12]:
X1 = np.array(X_char)
print(X1)
train_char_vectors = model.predict(X1)
        

[[63  9  3 ...  0  0  0]
 [11  9  3 ...  0  0  0]
 [ 3 24 46 ...  0  0  0]
 ...
 [24 71 25 ...  0  0  0]
 [71 63 63 ...  0  0  0]
 [30 64 71 ...  0  0  0]]


In [13]:
print(train_char_vectors.shape)
print("\n")
print(train_char_vectors)

(35178, 10)


[[-0.04015607  0.00966878 -0.02886212 ... -0.07019146  0.01894818
   0.00232804]
 [-0.04051049  0.00980142 -0.02905718 ... -0.03797981  0.0037691
   0.0265603 ]
 [-0.03947827  0.00945163 -0.02845969 ... -0.04789339  0.00932495
  -0.01452854]
 ...
 [-0.03824511  0.00914548 -0.02759524 ... -0.00695662  0.01212774
   0.01093699]
 [-0.04052166  0.00978761 -0.02908666 ... -0.06294253  0.03424528
  -0.00146282]
 [-0.03737618  0.00878344 -0.02716619 ... -0.04566007  0.03095722
  -0.02826047]]


In [20]:
def pred_new_num_enc(s) :
    
    pred = s

    l = []
    ip = []
    k=0

    for i in range(len(pred)):
        l.append(char2idx.get(pred[i]))
    #print(l)
    #print(char2idx.get("PAD"))        


    length = len(l)
    #print(length)

    if length < max_word_length :
        k = max_word_length - length
        print(k)

        for i in range (0,k) :
            l.append(char2idx.get("PAD"))    

    else :
        l = l[:max_word_length]
    #ip.append(l)

    l = np.array([l])
    #print(l)
    
    return l

In [21]:
def pred_new_vec() :
    train_char_vectors = model.predict(l)

    #print(train_char_vectors)
    
    return train_char_vectors

In [23]:
a1 = pred_new_num_enc("boy")
a2 = pred_new_num_enc("man")

print(cosine_similarity(a1 , a2))

22
22
[[0.45327704]]


In [24]:
a3 = pred_new_num_enc("apple")
a4 = pred_new_num_enc("banana")

print(cosine_similarity(a3 , a4))

20
19
[[0.66866157]]


In [25]:
a5 = pred_new_num_enc("girl")
a6 = pred_new_num_enc("woman")

print(cosine_similarity(a5 , a6))

21
20
[[0.31555707]]


In [27]:
a7 = pred_new_num_enc("cricket")
a8 = pred_new_num_enc("football")

print(cosine_similarity(a7 , a8))

18
17
[[0.59690781]]


In [28]:
a9 = pred_new_num_enc("laptop")
a10 = pred_new_num_enc("cmputer")

print(cosine_similarity(a9 , a10))

19
18
[[0.63692091]]
